In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/drive/MyDrive/COVID-19_Radiography_Dataset.rar').extractall('/content/sample_data')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
splitfolders.ratio('/content/sample_data/COVID-19_Radiography_Dataset', output="/content/sample_data", seed=1337, ratio=(.8, 0.2))

Copying files: 21165 files [00:04, 4751.81 files/s]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Flatten,Dense,Dropout

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/sample_data/train',labels='inferred',
    label_mode='int',batch_size=32,image_size=(256,256))
test_ds = keras.utils.image_dataset_from_directory(
    directory='/content/sample_data/val',labels='inferred',
    label_mode='int',batch_size=32,image_size=(256,256))

Found 16930 files belonging to 4 classes.
Found 4235 files belonging to 4 classes.


In [ ]:
def process(image, label):
  image = tf.cast(image/255.0, tf.float32)
  return image, label

In [ ]:
train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [ ]:
conv_part= VGG16(include_top=False,weights='imagenet',input_shape=(256,256,3))


58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
model = Sequential()
model.add(conv_part)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(4,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,validation_data=test_ds,epochs=25,batch_size=32)

Epoch 1/25
530/530 [==============================] - 278s 512ms/step - loss: 0.4352 - accuracy: 0.8419 - val_loss: 0.3428 - val_accuracy: 0.8767
Epoch 2/25
530/530 [==============================] - 276s 519ms/step - loss: 0.3450 - accuracy: 0.8751 - val_loss: 0.3360 - val_accuracy: 0.8819
Epoch 3/25
530/530 [==============================] - 273s 513ms/step - loss: 0.3346 - accuracy: 0.8800 - val_loss: 0.3194 - val_accuracy: 0.8904
Epoch 4/25
530/530 [==============================] - 272s 512ms/step - loss: 0.3257 - accuracy: 0.8859 - val_loss: 0.3179 - val_accuracy: 0.8829
Epoch 5/25
530/530 [==============================] - 272s 511ms/step - loss: 0.2890 - accuracy: 0.8934 - val_loss: 0.2996 - val_accuracy: 0.8942
Epoch 6/25
530/530 [==============================] - 274s 517ms/step - loss: 0.2859 - accuracy: 0.8966 - val_loss: 0.3180 - val_accuracy: 0.8869
Epoch 7/25
530/530 [==============================] - 273s 513ms/step - loss: 0.3009 - accuracy: 0.8959 - val_loss: 0.3098 -

In [ ]:
model.save("temp.h5")